# Crawl Questions from web

In [1]:
WEB_URL = 'http://zbranekvalitne.cz/zbrojni-prukaz/testove-otazky'
IMG_BASE_URL = 'http://zbranekvalitne.cz/'

In [2]:
import requests
from bs4 import BeautifulSoup
import dataclasses

soup = BeautifulSoup(requests.get(WEB_URL).text)


In [3]:
from dataclasses import dataclass

@dataclass
class Question:
    id: int
    question: str
    options: dict
    correct: list
    image: str = None

In [4]:
questions_div = soup.find('div', {'id': 'questions'})
questions = []
for idx, question_segment in enumerate(questions_div.find_all('div', {'class': 'ui raised segment'})):
    question_rows= question_segment.find_all('div', {'class': 'row'})
    # Question
    if len(question_rows) == 4:
        # Question and 3 options
        question_image = None
        question_text = question_rows[0].text
        answ_idx = 1
    elif len(question_rows) == 5:
        # Image, question and 3 options
        question_image = question_rows[0].find('img')['src']
        question_text = question_rows[1].text
        answ_idx = 2
    # Downlaod image
    if question_image:
        image_url = IMG_BASE_URL + question_image
        image_data = requests.get(image_url)
        image_format = image_url.split('.')[-1]
        question_image = f'{idx}.{image_format}'
        with open(f'data/output/images/{question_image}', 'wb') as f:
            f.write(image_data.content)
    # Answers
    options = {}
    correct_option = 'a'
    for ans_letter, ans_div in zip(['a', 'b', 'c'],question_rows[answ_idx:]):
        if 'correct-answer' in ans_div['class']:
            correct_option = ans_letter
        options[ans_letter] = ans_div.find('p', {'class': 'fifteen wide column'}).text[:-1]
    questions.append(Question(id=idx, question=question_text, options=options, correct=correct_option, image=question_image))

In [5]:
import json
with open('data/output/questions.json', 'w', encoding='utf8') as f:
    json.dump([dataclasses.asdict(x) for x in questions], f, indent=2, ensure_ascii=False)